# 🛍️ | Cora-For-Zava: 테스트 데이터셋 시뮬레이션

환영합니다! 이 노트북에서는 Azure AI Search와 Azure AI 평가 시뮬레이터를 사용하여 합성 데이터셋을 생성하는 방법을 안내합니다.

## 🛒 Zava 시나리오

**Cora**는 DIY 애호가를 위한 홈 인테리어 용품을 판매하는 가상의 소매업체 **Zava**의 고객 서비스 챗봇입니다. Cora가 하드웨어 및 홈 인테리어 제품에 대해 정확하고 유용한 답변을 제공하려면 품질 좋은 테스트 데이터가 필요합니다. 이 노트북은 제품 카탈로그를 기반으로 합성 질의-응답 쌍을 생성하여 배포 전 Cora의 성능을 측정할 수 있는 강력한 평가 데이터셋을 만드는 방법을 안내합니다.

## 🎯 완성 목표

이 노트북을 마치면 다음을 달성하게 됩니다.
- ✅ 제품 카탈로그에서 생성된 가상 쿼리-응답 쌍
- ✅ 평가를 위한 JSON Lines 형식의 데이터 세트 생성
- ✅ Azure AI 평가 시뮬레이터 사용 방법 학습
- ✅ 향후 평가 연습에 사용할 수 있도록 데이터 세트 저장

## 💡 학습 내용

- Azure AI 평가 시뮬레이터 구성 방법
- Azure AI Search에 연결하여 콘텐츠 검색 방법
- RAG 애플리케이션 콜백 생성 방법
- 가상 쿼리-응답 쌍 생성 방법
- 생성된 데이터 세트 저장 및 검토 방법

평가 데이터 세트를 생성할 준비가 되셨나요? 시작해 봅시다! 🚀

---

## Step 1: 환경 변수 확인

다음 환경 변수들은 이전 설정 단계에서 `.env` 파일에 이미 구성되어 있어야 합니다:

- **AZURE_OPENAI_API_KEY**: Azure OpenAI API 키
  - 제거하고 SystemManagedIdentity 사용 예정
- **AZURE_OPENAI_ENDPOINT**: Azure OpenAI 서비스 엔드포인트
- **AZURE_OPENAI_API_VERSION**: 사용할 API 버전
- **AZURE_OPENAI_DEPLOYMENT**: 배포한 대화형 모델 이름
- **AZURE_SEARCH_ENDPOINT**: Azure AI Search 서비스 엔드포인트
- **AZURE_SEARCH_API_KEY**: Azure AI Search API 키
- **AZURE_SEARCH_INDEX_NAME**: 검색 인덱스 이름

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify all required Azure service credentials are available
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_DEPLOYMENT",
    "AZURE_SEARCH_ENDPOINT",
    "AZURE_SEARCH_API_KEY",
    "AZURE_SEARCH_INDEX_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    raise EnvironmentError(
        f"❌ Missing environment variables: {', '.join(missing_vars)}")

print("✅ All environment variables configured!")

✅ All environment variables configured!


In [2]:
# Initialize Azure AI Search connection parameters
search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
search_api_key = os.environ.get("AZURE_SEARCH_API_KEY")
search_index_name = os.environ.get("AZURE_SEARCH_INDEX_NAME")

print(f"✅ Azure AI Search configured with index: {search_index_name}")

✅ Azure AI Search configured with index: zava-products


## Step 2: Azure OpenAI 모델 구성

이제 시뮬레이터가 합성 질의와 응답을 생성할 때 사용할 모델 구성을 만듭니다.

In [3]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from azure.identity import DefaultAzureCredential

# Create model configuration using environment variables
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    credential=DefaultAzureCredential(),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)


print(f"✅ Model configuration created for: {model_config['azure_deployment']}")

✅ Model configuration created for: gpt-4.1


## Step 3: 시뮬레이터 초기화

모델 구성을 사용해 Azure AI Evaluation Simulator 인스턴스를 생성합니다.

In [4]:
# 4. Initialize the Azure AI Evaluation Simulator
from azure.ai.evaluation.simulator import Simulator

# Create simulator instance with the configured model
simulator = Simulator(model_config=model_config)
print("📊 Simulator initialized successfully!")

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


📊 Simulator initialized successfully!


## Step 4: Search 함수 정의

이 함수는 질의를 기반으로 Azure AI Search 인덱스에서 관련 콘텐츠를 검색합니다. RAG 애플리케이션 콜백에서 컨텍스트를 검색할 때 사용됩니다.

In [5]:
# 5. Define function to search Azure AI Search index and retrieve content
import requests
import json


def search_index_for_content(query: str, top_k: int = 5) -> str:
    """
    Search the Azure AI Search index for relevant content based on a query.

    Args:
        query (str): Search query to find relevant content
        top_k (int): Number of top results to retrieve (default: 5)

    Returns:
        str: Combined content from search results, truncated to 1000 characters
    """
    # Construct the search API endpoint
    search_url = f"{search_endpoint}/indexes/{search_index_name}/docs/search?api-version=2023-11-01"

    # Set up request headers with API key authentication
    headers = {
        "Content-Type": "application/json",
        "api-key": search_api_key
    }

    # Define the search query payload
    search_payload = {
        "search": query,
        "top": top_k,
        "select": "content,title"  # Updated to match available fields
    }

    try:
        # Execute the search request
        response = requests.post(
            url=search_url, headers=headers, json=search_payload)
        response.raise_for_status()  # Raise an exception for HTTP error codes

        # Parse the search results
        search_results = response.json()
        combined_content = ""

        # Extract and combine content from search results
        for result in search_results.get("value", []):
            # Prioritize 'content' field, fall back to 'title'
            content = result.get("content") or result.get("title", "")
            if content:
                combined_content += content + " "

        # Limit content length to prevent token limits and improve performance
        return combined_content[:1000].strip()

    except requests.exceptions.RequestException as e:
        print(f"❌ Error searching index: {e}")
        return f"Error retrieving content for query: {query}"
    except json.JSONDecodeError as e:
        print(f"❌ Error parsing search response: {e}")
        return f"Error processing search results for query: {query}"

## Step 5: Search 함수 테스트

샘플 질의로 Search 함수가 올바르게 작동하는지 확인합니다.

In [6]:
# Test the search function with a sample query
test_query = "eggshell paint"
retrieved_content = search_index_for_content(test_query)

print(f"✅ Search test successful!")
print(f"📄 Retrieved response: {retrieved_content} for query: '{test_query}'")

✅ Search test successful!
📄 Retrieved response: Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup. Disposable plastic liners that fit standard paint trays for quick cleanup and color changes. Environmentally friendly zero-VOC paint for healthy indoor air quality in all living spaces. Premium acrylic exterior paint with fade resistance and superior adhesion for long-lasting protection. Washable semi-gloss interior paint for kitchens, bathrooms, and trim work with moisture resistance. for query: 'eggshell paint'


## Step 6: RAG 애플리케이션 콜백 생성

이 콜백 함수는 RAG 애플리케이션을 시뮬레이션합니다:
- 메시지에서 사용자 질의 추출
- 인덱스에서 관련 컨텍스트 검색
- Azure OpenAI를 사용해 응답 생성
- 시뮬레이터에 필요한 형식으로 응답 반환

In [7]:
# 7. Define the application callback function for the simulator
from typing import Dict, Any, Optional
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


async def rag_application_callback(
    messages: Dict,
    stream: bool = False,
    session_state: Any = None,
    context: Optional[Dict[str, Any]] = None,
) -> Dict:
    """
    Callback function that simulates a RAG (Retrieval-Augmented Generation) application.

    This function:
    1. Extracts the user query from the message
    2. Searches the Azure AI Search index for relevant content
    3. Uses Azure OpenAI to generate a response based on the retrieved content
    4. Returns the response in the expected format

    Args:
        messages (Dict): Message history containing user queries
        stream (bool): Whether to stream the response (not used in this implementation)
        session_state (Any): Session state information
        context (Optional[Dict[str, Any]]): Additional context information

    Returns:
        Dict: Response containing the generated message and metadata
    """
    # Extract the user's query from the latest message
    messages_list = messages["messages"]
    user_query = messages_list[-1]["content"]

    # Initialize Azure OpenAI client
    openai_client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        azure_ad_token_provider=get_bearer_token_provider(
            DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    )

    # Retrieve relevant content from the search index
    retrieved_context = search_index_for_content(user_query)

    # Create a system prompt that instructs the model to use the retrieved context
    system_prompt = """You are a polite and helpful assistant that answers questions based on the provided context. 
Use the context information to provide accurate and relevant responses. If the context doesn't contain 
enough information to answer the question, say so politely. If the context mentions a product by name, reference it in the response."""

    # Generate response using Azure OpenAI
    try:
        completion = openai_client.chat.completions.create(
            model=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Context: {retrieved_context}"},
                {"role": "user", "content": f"Question: {user_query}"}
            ],
            max_tokens=500,
            temperature=0.7,
        )

        # Extract the generated response
        ai_response = completion.choices[0].message.content

    except Exception as e:
        ai_response = f"Sorry, I encountered an error while generating a response: {str(e)}"

    # Format the response according to the expected structure
    response_message = {
        "content": ai_response,
        "role": "assistant",
        "context": retrieved_context,
    }

    # Add the response to the message history
    messages["messages"].append(response_message)

    # Return the complete response structure
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": retrieved_context
    }

## Step 7: 합성 데이터 생성

이제 시뮬레이터를 사용해 검색 인덱스의 콘텐츠를 기반으로 합성 질의-응답 쌍을 생성합니다. 여기서는 평가를 위해 5개의 질의-응답 쌍을 생성합니다.

In [8]:
# Verify the retrieved content is ready for simulation
if not retrieved_content:
    raise ValueError(
        "No content retrieved from search. Please run Step 5 again with a valid query.")

print(
    f"✅ Ready to simulate with {len(retrieved_content)} characters of content")

✅ Ready to simulate with 492 characters of content


In [9]:
from pathlib import Path

# Run the simulator to generate synthetic data
# The simulator needs a list of query-response dictionaries or plain text
synthetic_outputs = await simulator(
    target=rag_application_callback,
    text=retrieved_content,
    num_queries=10,
    max_conversation_turns=1,
)

print(f"✅ Generated {len(synthetic_outputs)} synthetic query-response pairs!")

/home/vscode/.local/lib/python3.12/site-packages/azure/ai/evaluation/simulator/_simulator.py:157: UserWarning: You have specified 'num_queries' > len('tasks') (10 > 0). All tasks will be used for generation and the remaining 10 lines will be simulated in task-free mode
  warnings.warn(
Generating: 100%|██████████████████████████████████████████████| 10/10 [00:40<00:00,  4.05s/message]

✅ Generated 10 synthetic query-response pairs!


## Step 8: 데이터셋 파일로 저장

생성된 데이터셋을 JSONL 파일로 저장해 평가 워크플로에서 사용합니다.

In [10]:
output_file = Path("21-simulate-datasets.jsonl")

# Write each output as a JSON line to the file
with output_file.open("w") as f:
    for output in synthetic_outputs:
        f.write(output.to_eval_qr_json_lines())

print(f"✅ Dataset saved to: {output_file.absolute()}")

✅ Dataset saved to: /workspaces/ignite25-kor-observe-manage-and-scale-agentic-ai-apps-with-microsoft-foundry/labs/2-models/21-simulate-datasets.jsonl


## Step 9: 데이터셋 미리보기

생성된 데이터셋을 로드해 구조와 품질을 확인합니다.

In [11]:
import pandas as pd

# Load and display the dataset
dataset_df = pd.read_json(output_file, lines=True)

print(
    f"✅ Dataset loaded: {dataset_df.shape[0]} rows, {dataset_df.shape[1]} columns")
print(f"📋 Columns: {list(dataset_df.columns)}")
print()

# Show first few records
dataset_df.head(10)

✅ Dataset loaded: 10 rows, 3 columns
📋 Columns: ['query', 'response', 'context']



,query,response,context
0,What type of finish does the durable eggshell ...,The durable eggshell finish paint has a subtle...,Durable eggshell finish paint with subtle shee...
1,Which paint is described as ideal for living r...,The durable eggshell finish paint with subtle ...,Durable eggshell finish paint with subtle shee...
2,What product fits standard paint trays for qui...,The product that fits standard paint trays for...,Disposable plastic liners that fit standard pa...
3,What type of paint is recommended for healthy ...,The environmentally friendly zero-VOC paint is...,Environmentally friendly zero-VOC paint for he...
4,Which paint is described as environmentally fr...,The paint described as environmentally friendl...,Environmentally friendly zero-VOC paint for he...
5,What is the main feature of premium acrylic ex...,The main feature of premium acrylic exterior p...,Premium acrylic exterior paint with fade resis...
6,Which paint offers superior adhesion for long-...,The premium acrylic exterior paint offers supe...,Premium acrylic exterior paint with fade resis...
7,What type of interior paint is recommended for...,"The recommended interior paint for kitchens, b...",Washable semi-gloss interior paint for kitchen...
8,Which paint provides moisture resistance for k...,The washable semi-gloss interior paint provide...,Washable semi-gloss interior paint for kitchen...
9,What is a key feature of the washable semi-glo...,A key feature of the washable semi-gloss inter...,Washable semi-gloss interior paint for kitchen...


## Step 10: 데이터셋 파일 검토

`21-simulate-datasets.jsonl` 파일을 VS Code에서 열어 생성된 질의-응답-컨텍스트 트리플을 확인하고 데이터 품질을 검토하세요.

## 다음 단계

합성 평가 데이터셋을 성공적으로 생성했습니다! 이제 다음을 수행할 수 있습니다:
- RAG 애플리케이션의 검색 품질 평가
- 시스템 프롬프트 테스트 및 파인튜닝
- 애플리케이션 성능 벤치마크
- 개발용 테스트 케이스 생성

수고하셨습니다! 🎉